In [1]:
import numpy as np
import torch
from torch import utils
import pandas as pd
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch import nn 
from torch.nn import functional as F
import pytorch_lightning as pl
from matplotlib import cm
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from scipy import signal as sig
import os
from pathlib import Path
import re
from torch.utils import data
import random
import pandas as pd
import pickle
import numpy as np
from pathlib import Path
from dataloader import LandmarkDataset, SequenceDataset, LandmarkWaveletDataset
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import normalized_mutual_info_score, confusion_matrix, accuracy_score

pd.set_option('mode.chained_assignment', None)
plt.rcParams['svg.fonttype'] = 'none'
%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.random.manual_seed(SEED)

In [4]:
data_root = Path('/mnt/Storage1/Data/K7')

# landmark_files = list(data_root.glob('2020-*/Down/*DeepCut*.h5')) # DeepLabCut landmarks
landmark_files = list(data_root.glob('2020-*/Down/model=*.h5')) # HourGlass landmarks

landmark_files

[PosixPath('/mnt/Storage1/Data/K7/2020-08-10/Down/model=5_27_10_29_20-video=0041.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-04/Down/model=5_27_10_29_20-video=0037.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-16/Down/model=5_27_10_29_20-video=0047.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-12/Down/model=5_27_10_29_20-video=0044.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-13/Down/model=5_27_10_29_20-video=0045.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-06/Down/model=5_27_10_29_20-video=0039.h5'),
 PosixPath('/mnt/Storage1/Data/K7/2020-08-05/Down/model=5_27_10_29_20-video=0038.h5')]

In [5]:
# define model and dataloader, load saved model

from simple_autoencoder import Autoencoder, PLAutoencoder
from dataloader import LandmarksDataModule
seqlen = 60
to_drop = None
dm = LandmarksDataModule(landmark_files, seqlen=seqlen, step=1, to_drop=to_drop)
dm.prepare_data()
n_parts = len(dm.data_frames[0].columns.levels[0])
model = PLAutoencoder(landmark_files, n_neurons=[2*n_parts*seqlen, 1024, 512, 512, 10], lr=5e-4, patience=20, dropout=0.1)
model_dir = Path('models/HG_landmarks3')
model.load_state_dict(torch.load(model_dir / 'model.pt'))

<All keys matched successfully>

## Cluster the data

cluster the dataset using KMeans on the embedded space.
 
 It is possible to filter out timesteps with less than some energy level - to filter out segments with no or little movement, with the parameter filter_by_energy.

In [10]:
# calculates energies of each sliding window in the dataset, can be used for filtering.
def calculate_energies(dm):
    energies = np.zeros(len(dm.all_ds))
    for i, item in enumerate(dm.all_ds):
        item = item.reshape((dm.seqlen, -1))
        ff, Pxx = sig.periodogram(item.T, fs=dm.fps)
        energies[i] = Pxx.T[:10].mean()


filter_by_energy = False


if filter_by_energy:
    X_encoded = model.model.encode(dm.all_ds)
    energiess = calcuate_energies(dm)
    selected_idxs = np.where(energies > 5e-5)[0]
    idx2orig = {idx : i for i, idx in enumerate(selected_idxs)}
    K = model.model.encoder[-1].out_features
    kmeans = KMeans(K)
    labels = kmeans.fit_predict(X_encoded[selected_idxs])
    _labels = np.zeros(len(X_encoded), dtype=np.int32) - 2.  # set labels with -2, meaning "filtered out"
    for i, lbl in enumerate(labels):
        _labels[selected_idxs[i]] = lbl # set every non filtered timestep with the appropriate label.
    labels = _labels
else:
    X_encoded = model.model.encode(dm.all_ds)
    K = model.model.encoder[-1].out_features
    kmeans = KMeans(K)
    labels = kmeans.fit_predict(X_encoded)

array([     0,      1,      2, ..., 961664, 961665, 961666])

## Segment the data into cluster segments

Divide the landmarks timeseries to segments such that each segment is a sequence of timesteps with same cluster label.
Filter the segments by length, such that segments shorter than 30 timesteps (0.25 seconds) would not be included.

Each segment is a triplet of (start_timestep, duration (in timesteps), cluster_label)

In [10]:
import dataloader

# find the labels for each video file and put in the dictionary
# labels_dict = {}
# for df in dm.raw_data:
#     labels_dict[df.attrs['file']] = np.zeros(len(df), dtype=np.int32) - 1
    
for df in dm.raw_data:
    df['label'] = labels_dict[df.attrs['file']]
    
segments_dict = {}
for file, labels in labels_dict.items():
    split_at = np.where(np.diff(labels) != 0)[0] + 1
    split_at = np.append(np.zeros(1, dtype=np.int), np.where(np.diff(labels) != 0)[0] + 1)
    segments = [(split_at[i-1], split_at[i] - split_at[i-1], labels[split_at[i-1]]) for i in range(1, len(split_at))]
    segments = [seg for seg in segments if seg[-1] >= 0]
    segments = [seg for seg in segments if seg[1] >= 30]
    segments_dict[file] = segments

In [ ]:
import pickle
with open(model_dir / 'kmeans.pkl', 'wb') as file:
    pickle.dump(kmeans, file)
    
np.save(model_dir / 'labels.np', labels)

with open(model_dir / 'labels_dict.pkl', 'wb') as file:
    pickle.dump(labels_dict, file)
    
with open(model_dir / 'data_dict.pkl', 'wb') as file:
    pickle.dump(data_dict, file)
    
with open(model_dir / 'segments_dict.pkl', 'wb') as file:
    pickle.dump(segment_dict, file)
    
with open(model_dir / 'x_encoded_dict.pkl', 'wb') as file:
    pickle.dump(X_encoded_dict, file)

## Save clips of clusters

for each cluster, sample some segments from the cluster and save them as clips in a folder.

In [12]:
import cv2 as cv
import tkinter as tk
from triplets import landmarks_video, triplets_gui

cluster_dir = Path("/mnt/Storage1/shuki/projects/clusters/HG_landmarks_clusters")

# write the clip to a video file
def write_video(vid, file, fps, codec='mp4v'):
    n_frames, width, height, _ = vid.shape
    fourcc = cv.VideoWriter_fourcc(*codec)
    writer = cv.VideoWriter(str(file), fourcc, fps, (height, width), True)
    for frame in vid:
        writer.write(frame)
    writer.release()

frame_to_time = lambda idx: f'{idx // (video.fps*60)}_{(idx % (video.fps*60)) // video.fps}'

os.makedirs(cluster_dir, exist_ok=True)

for file in dm.landmark_files:
    video = landmarks_video.LandmarksVideo(file.parent)
    file_id = file.name[:4]
    segments = segments_dict[file]
    for start_frame, n_frames, lbl in segments:
        mid_frame = start_frame + n_frames // 2
        n_frames = min(120 * 2, n_frames) # max 240 frames in a clip
        start, end = mid_frame - n_frames // 2 , mid_frame + n_frames // 2
        save_dir = cluster_dir / f'{lbl}'
        os.makedirs(save_dir, exist_ok=True)
        clip = video[start: end: 2] # 
        video_file_name = f'{file_id}_{frame_to_time(mid_frame)}.avi'
        write_video(clip, save_dir / video_file_name, fps=24)